# This step combines some lines from Step 0 and Step 3
## It is designed to update the "cascade" graphs showing grad rate by seats

- Reference PPTX files for both before and after are in the Notebook directory

In [1]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import csv
import os
import io
import urllib

# Edit these to reflect any changes
work_location = 'inputs'
latest_year = 2019
directory_file = f'hd{latest_year}.csv'
gr_file = f'gr{latest_year}.csv'
output_file = f'grad{latest_year}.csv'

In [2]:
def grab_zipfile(baseUrl, fileroot):
    '''uses a known static baseUrl to grab a zipped archive'''
    full_url = baseUrl + fileroot + '.zip'
    print(full_url, flush=True)
    remoteFile = urllib.request.urlopen(full_url)

    bio = io.BytesIO()
    bio.write(remoteFile.read())

    z = ZipFile(bio, 'r')
    print(z.namelist(), flush=True)
    z.extract(z.namelist()[0])
    return z.namelist()[0]

In [3]:
os.chdir(work_location)

In [4]:
# Here's where the files live:
baseUrl = 'https://nces.ed.gov/ipeds/datacenter/data/'

# The master directory:
grab_zipfile(baseUrl, f'HD{latest_year}')

grab_zipfile(baseUrl, f'HD{latest_year}_DICT')

https://nces.ed.gov/ipeds/datacenter/data/HD2019.zip
['hd2019.csv']
https://nces.ed.gov/ipeds/datacenter/data/HD2019_DICT.zip
['hd2019.xlsx']


'hd2019.xlsx'

In [5]:
# Most recent grad data
fileroot = f'GR{latest_year}'

grab_zipfile(baseUrl, fileroot)
grab_zipfile(baseUrl, fileroot+'_DICT')

https://nces.ed.gov/ipeds/datacenter/data/GR2019.zip
['gr2019.csv']
https://nces.ed.gov/ipeds/datacenter/data/GR2019_DICT.zip
['gr2019.xlsx']


'gr2019.xlsx'

In [6]:
# We'll read in the grad rate file and add an AA_H column
gr_df = pd.read_csv(gr_file, index_col=['UNITID'],
                    usecols=['UNITID', 'GRTYPE', 'GRTOTLT','GRBKAAT','GRHISPT'],
                    na_values='.',
                    dtype={'GRTOTLT':float,'GRBKAAT':float,'GRHISPT':float},
                    encoding='latin-1')
gr_df.rename(columns={'GRTOTLT':'Total','GRBKAAT':'Black','GRHISPT':'Hisp'}, inplace=True)
gr_df['AA_H']=gr_df.Black+gr_df.Hisp
gr_df.head()

,GRTYPE,Total,Black,Hisp,AA_H
UNITID,,,,,
100654,2,928.0,867.0,11.0,878.0
100654,3,276.0,261.0,3.0,264.0
100654,4,342.0,317.0,5.0,322.0
100654,6,932.0,871.0,11.0,882.0
100654,7,4.0,4.0,0.0,4.0


In [7]:
# We now have to sort through these GRTYPES:
# 8 is the adjusted cohort for bachelor's seeking students (completions: 12=6yr, 13=4yr, 14=5yr; transfers=16)
# 29 for associate's seeking (completions: 30=3yr 35=2yr; transfers=33)
# We'll build a list of unitids that have both starting cohorts and completions for either one
valid_unitids = {}
valid_unitids = list( (set(gr_df[gr_df['GRTYPE']==8].index) & set(gr_df[gr_df['GRTYPE']==12].index)) |
                                (set(gr_df[gr_df['GRTYPE']==29].index) & set(gr_df[gr_df['GRTYPE']==30].index)) )
print(f"{len(gr_df)}, {len(valid_unitids)}")

50447, 3556


In [8]:
# We'll use the basic "hd" directory to form the base of the final year output
def create_year_df(df, source_df1, source_df2):
    """Apply function to pull the appropriate data into a single row per college"""
    ix = df.name
    if ix in source_df1.index:
        return source_df1.loc[ix][['Total','Black','Hisp','AA_H']]
    elif ix in source_df2.index:
        return source_df2.loc[ix][['Total','Black','Hisp','AA_H']]
    else:
        return [np.nan,np.nan,np.nan,np.nan]

dir_df = pd.read_csv(directory_file, index_col=['UNITID'],
                 usecols=['UNITID','INSTNM','ICLEVEL'],encoding='latin-1')
dir_df = dir_df[dir_df.index.isin(valid_unitids)]

# First do the completions
start1 = gr_df[gr_df.GRTYPE == 12]
start2 = gr_df[gr_df.GRTYPE == 30]
dir_df[['Cl_Total','Cl_Black','Cl_Hisp','Cl_AA_H']]=dir_df.apply(create_year_df,axis=1,result_type="expand",
                                                                args=(start1,start2))
# Then do the starts
start1 = gr_df[gr_df.GRTYPE == 8]
start2 = gr_df[gr_df.GRTYPE == 29]
dir_df[['St_Total','St_Black','St_Hisp','St_AA_H']]=dir_df.apply(create_year_df,axis=1,result_type="expand",
                                                                args=(start1,start2))
# Next the transfers
start1 = gr_df[gr_df.GRTYPE == 16]
start2 = gr_df[gr_df.GRTYPE == 33]
dir_df[['Xf_Total','Xf_Black','Xf_Hisp','Xf_AA_H']]=dir_df.apply(create_year_df,axis=1,result_type="expand",
                                                                args=(start1,start2))

# Add on-time completions
start1 = gr_df[gr_df.GRTYPE == 13]
start2 = gr_df[gr_df.GRTYPE == 35]
dir_df[['Cl4_Total','Cl4_Black','Cl4_Hisp','Cl4_AA_H']]=dir_df.apply(create_year_df,axis=1,result_type="expand",
                                                                args=(start1,start2))

# Add 5-year completions (using 3-yr for 2-year colleges)
start1 = gr_df[gr_df.GRTYPE == 14]
start2 = gr_df[gr_df.GRTYPE == 30]
dir_df[['Cl5_Total','Cl5_Black','Cl5_Hisp','Cl5_AA_H']]=dir_df.apply(create_year_df,axis=1,result_type="expand",
                                                                args=(start1,start2))

# Finally, calculated within year stats
for type in ['Total','Black','Hisp','AA_H']:
    dir_df['GR_'+type]=dir_df['Cl_'+type]/dir_df['St_'+type]
    dir_df['GR4_'+type]=dir_df['Cl4_'+type]/dir_df['St_'+type]
    dir_df['GR5_'+type]=dir_df['Cl5_'+type]/dir_df['St_'+type]
    dir_df['Xfr_'+type]=dir_df['Xf_'+type]/dir_df['St_'+type]
    dir_df['CI_'+type]=np.sqrt(dir_df['GR_'+type]*(1-dir_df['GR_'+type])/dir_df['St_'+type])
    dir_df.replace(np.inf,np.nan)
dir_df.head()

,INSTNM,ICLEVEL,Cl_Total,Cl_Black,Cl_Hisp,Cl_AA_H,St_Total,St_Black,St_Hisp,St_AA_H,...,GR_Hisp,GR4_Hisp,GR5_Hisp,Xfr_Hisp,CI_Hisp,GR_AA_H,GR4_AA_H,GR5_AA_H,Xfr_AA_H,CI_AA_H
UNITID,,,,,,,,,,,,,,,,,,,,,
100654,Alabama A & M University,1,276.0,261.0,3.0,264.0,928.0,867.0,11.0,878.0,...,0.272727,0.181818,0.090909,0.454545,0.134282,0.300683,0.078588,0.158314,0.366743,0.015475
100663,University of Alabama at Birmingham,1,1092.0,214.0,27.0,241.0,1725.0,384.0,40.0,424.0,...,0.675000,0.325000,0.275000,0.150000,0.074057,0.568396,0.304245,0.209906,0.247642,0.024054
100690,Amridge University,1,1.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000
100706,University of Alabama in Huntsville,1,368.0,30.0,5.0,35.0,638.0,70.0,15.0,85.0,...,0.333333,0.200000,0.133333,0.533333,0.121716,0.411765,0.105882,0.247059,0.411765,0.053381
100724,Alabama State University,1,497.0,425.0,8.0,433.0,1517.0,1380.0,15.0,1395.0,...,0.533333,0.333333,0.200000,0.333333,0.128812,0.310394,0.142652,0.115412,0.379928,0.012387


In [9]:
# Here, we'll save for reference
dir_df.to_csv(output_file, na_rep="N/A")

## Now, we'd like to create some structured data to be used in Excel plots
- This will show seats by grad rate percentage, first collectively and second for only AA/H students
- the first chart we're interested in looks like this (reference PPTX in the Notebook directory):
![First slide](gr_image1.png)


### There are x numbers to grab for the above slide:
- Start year and end year: use the "Dict" file or gr2019.xlsx and find the "cohort year" for 4-year students. In this case, it was 2013; the end year is then 2013+6=2019
- Total number of freshmen seeking bachelor's: this is the sum of St_Total for all colleges with IC_LEVEL=1 (calculated below)
- Completion rate: sum of Cl_Total divided by the sum of St_Total

In [10]:
df = dir_df[dir_df.ICLEVEL==1]

In [11]:
s_val = "St_Total"
c_val = "Cl_Total"
total_sums = df[[s_val,c_val]].sum()
print(f"Total starts is {total_sums[s_val]/1000000:.2f}M or {total_sums[s_val]:.0f}")
print(f"Completion rate: {total_sums[c_val]/total_sums[s_val]*100:.1f}% with {total_sums[c_val]:.0f} completers")

Total starts is 1.50M or 1495354
Completion rate: 63.0% with 942480 completers


### Some notes on updating the PowerPoint slide
- The X-axis is displayed by PowerPoint like a date; take the total starts number above and display it as a date in an Excel cell to see what the upper bound should be for display
- The two integers above should be pasted into the yellow area of the chart data when viewing "data in Excel" to update the line

---

### the second chart we're interested in looks like this (reference PPTX in the Notebook directory):
![Second slide](gr_image2.png)

- Note that this slide several numbers that simply repeat from the previous one; use those numbers again
- Here are the updated 4/5/6 year split numbers:

In [15]:
s_val = "St_Total"
c_val = "Cl_Total"
c4_val = "Cl4_Total"
c5_val = "Cl5_Total"
total_sums = df[[s_val, c_val, c4_val, c5_val]].sum()
print(f"Total starts is {total_sums[s_val]/1000000:.2f}M or {total_sums[s_val]:.0f}")
print(f"Completion rate: {total_sums[c_val]/total_sums[s_val]*100:.1f}% with {total_sums[c_val]:.0f} completers")
print(f"4-year Completion rate: {total_sums[c4_val]/total_sums[s_val]*100:.1f}% with {total_sums[c4_val]:.0f} completers")
print(f"5-year Completion rate: {total_sums[c5_val]/total_sums[s_val]*100:.1f}% with {total_sums[c5_val]:.0f} completers")
print(f"6-year only Completion rate: {(total_sums[c_val]-total_sums[c4_val]-total_sums[c5_val])/total_sums[s_val]*100:.1f}% with {(total_sums[c_val]-total_sums[c4_val]-total_sums[c5_val]):.0f} completers")

Total starts is 1.50M or 1495354
Completion rate: 63.0% with 942480 completers
4-year Completion rate: 44.7% with 668900 completers
5-year Completion rate: 14.7% with 220009 completers
6-year only Completion rate: 3.6% with 53571 completers


---

### the third chart we're interested in looks like this (reference PPTX in the Notebook directory):
![Third slide](gr_image3.png)

- Note that this slide several numbers that simply repeat from the previous one; use those numbers again
- The two "Noble" callouts are calculated elsewhere (the staff one is from ~2015)
- The main detail needed here is a spreadsheet with the data points for each percentile; those will be generated by the following code below; it will have the following format:

|(blank)|Height1|Height2|Width|
|--------------|-------|-------|-----|
|(blank)|0|0|0|
|0%|0|0|0|
|(blank)|1|205|0|
|1%|1|205|206|
|(blank)|58|3005|206|
|2%|58|3005|3269|
|...|...|...|...|
|100%|173|0|1487685|
|(blank)|0|0|148765|

Basically, this version of Excel graph has two rows per data point that are used to "draw" the graph shown above
- Height1 is the number of completers for the given percentage
- Height2 is the number of non-completers for the given percentage
- Width is the x position for a point; each percentage has a pair of row where the first value is the cumulative sum of all prior points while the second value is the cumulative with this current percentage added

In [21]:
# This function will create the slice data by number of seats for a set of variables
def slice_constructor(df, gr_var, start_var, complete_var):
    # gr_var = the label for the grad rate we're slicing by
    # start_var = the label for the "Start" value used to rate the institution
    ret = [["", "Height1", "Height2", "Width"],
           ["", 0, 0, 0],
           ["0%", 0, 0, 0]]
    cum_count = 0
    for i in range(100):
        lb = i/100.
        ub = (i+1)/100.
        cur_df = df[(df[gr_var] > lb) & (df[gr_var] <= ub)]
        sum_df = cur_df[[start_var, complete_var]].sum()
        starters = sum_df[start_var]
        completers = sum_df[complete_var]
        non_completers = starters - completers
        ret.extend([
            ["", int(completers), int(non_completers), int(cum_count)],
            [f"{i+1}%", int(completers), int(non_completers), int(cum_count + starters)]
        ])
        cum_count += starters
        if i == 49:
            lte_50_count = cum_count
        if i == 79:
            lte_80_count = cum_count
            
    ret.extend([["", 0, 0, int(cum_count)]])
    
    return (
        ret,
        lte_50_count,
        lte_80_count,
        cum_count
    )

slide_lol, lte_50_count, lte_80_count, all_count = slice_constructor(df, "GR_Total", "St_Total", "Cl_Total")

with open("total_slices.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(slide_lol)

print(f"{lte_50_count:.0f} or {lte_50_count/all_count*100:.2f}% of seats below 50%")
print(f"{lte_80_count:.0f} or {lte_80_count/all_count*100:.2f}% of seats below 80%")

352999 or 23.61% of seats below 50%
1169286 or 78.19% of seats below 80%


---

### the fourth through sixth charts we're interested in looks like this (reference PPTX in the Notebook directory):
![Fourth slide](gr_image4.png)
![Fifth slide](gr_image5.png)
![Sixth slide](gr_image6.png)

- Note that these are basically recuts by race, but holding the x-axis the same

In [22]:
# First the fourth chart
s_val = "St_AA_H"
c_val = "Cl_AA_H"
total_sums = df[[s_val,c_val]].sum()
print(f"Total starts is {total_sums[s_val]/1000000:.2f}M or {total_sums[s_val]:.0f}")
print(f"Completion rate: {total_sums[c_val]/total_sums[s_val]*100:.1f}% with {total_sums[c_val]:.0f} completers")

Total starts is 0.36M or 359177
Completion rate: 50.7% with 182245 completers


In [23]:
# Next the fifth chart
s_val = "St_AA_H"
c_val = "Cl_AA_H"
c4_val = "Cl4_AA_H"
c5_val = "Cl5_AA_H"
total_sums = df[[s_val, c_val, c4_val, c5_val]].sum()
print(f"Total starts is {total_sums[s_val]/1000000:.2f}M or {total_sums[s_val]:.0f}")
print(f"Completion rate: {total_sums[c_val]/total_sums[s_val]*100:.1f}% with {total_sums[c_val]:.0f} completers")
print(f"4-year Completion rate: {total_sums[c4_val]/total_sums[s_val]*100:.1f}% with {total_sums[c4_val]:.0f} completers")
print(f"5-year Completion rate: {total_sums[c5_val]/total_sums[s_val]*100:.1f}% with {total_sums[c5_val]:.0f} completers")
print(f"6-year only Completion rate: {(total_sums[c_val]-total_sums[c4_val]-total_sums[c5_val])/total_sums[s_val]*100:.1f}% with {(total_sums[c_val]-total_sums[c4_val]-total_sums[c5_val]):.0f} completers")

Total starts is 0.36M or 359177
Completion rate: 50.7% with 182245 completers
4-year Completion rate: 29.6% with 106472 completers
5-year Completion rate: 16.1% with 57773 completers
6-year only Completion rate: 5.0% with 18000 completers


In [26]:
# Now the 6th chart:
slide_lol_aah, lte_50_count_aah, lte_80_count_aah, all_count_aah = slice_constructor(df, "GR_AA_H", "St_AA_H", "Cl_AA_H")

with open("aah_slices.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(slide_lol_aah)

print(f"{lte_50_count_aah:.0f} or {lte_50_count_aah/all_count_aah*100:.2f}% of seats below 50%")
print(f"{lte_80_count_aah:.0f} or {lte_80_count_aah/all_count_aah*100:.2f}% of seats below 80%")

176929 or 49.34% of seats below 50%
330691 or 92.23% of seats below 80%


---
### BONUS: New this time around, we'll add stats for Black students specifically

In [27]:
# Fourth chart repeated, Black only
s_val = "St_Black"
c_val = "Cl_Black"
total_sums = df[[s_val,c_val]].sum()
print(f"Total starts is {total_sums[s_val]/1000000:.2f}M or {total_sums[s_val]:.0f}")
print(f"Completion rate: {total_sums[c_val]/total_sums[s_val]*100:.1f}% with {total_sums[c_val]:.0f} completers")

Total starts is 0.16M or 158026
Completion rate: 44.2% with 69894 completers


In [28]:
# Next the fifth chart, Black only
s_val = "St_Black"
c_val = "Cl_Black"
c4_val = "Cl4_Black"
c5_val = "Cl5_Black"
total_sums = df[[s_val, c_val, c4_val, c5_val]].sum()
print(f"Total starts is {total_sums[s_val]/1000000:.2f}M or {total_sums[s_val]:.0f}")
print(f"Completion rate: {total_sums[c_val]/total_sums[s_val]*100:.1f}% with {total_sums[c_val]:.0f} completers")
print(f"4-year Completion rate: {total_sums[c4_val]/total_sums[s_val]*100:.1f}% with {total_sums[c4_val]:.0f} completers")
print(f"5-year Completion rate: {total_sums[c5_val]/total_sums[s_val]*100:.1f}% with {total_sums[c5_val]:.0f} completers")
print(f"6-year only Completion rate: {(total_sums[c_val]-total_sums[c4_val]-total_sums[c5_val])/total_sums[s_val]*100:.1f}% with {(total_sums[c_val]-total_sums[c4_val]-total_sums[c5_val]):.0f} completers")

Total starts is 0.16M or 158026
Completion rate: 44.2% with 69894 completers
4-year Completion rate: 25.7% with 40568 completers
5-year Completion rate: 14.5% with 22912 completers
6-year only Completion rate: 4.1% with 6414 completers


In [29]:
# Now the 6th chart, Black only:
slide_lol_b, lte_50_count_b, lte_80_count_b, all_count_b = slice_constructor(df, "GR_Black", "St_Black", "Cl_Black")

with open("Black_slices.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(slide_lol_b)

print(f"{lte_50_count_b:.0f} or {lte_50_count_b/all_count_b*100:.2f}% of seats below 50%")
print(f"{lte_80_count_b:.0f} or {lte_80_count_b/all_count_b*100:.2f}% of seats below 80%")

100478 or 63.80% of seats below 50%
149763 or 95.10% of seats below 80%


In [30]:
# Finally, for reference, the 4/5/6 stats for Latinx students:
s_val = "St_Hisp"
c_val = "Cl_Hisp"
c4_val = "Cl4_Hisp"
c5_val = "Cl5_Hisp"
total_sums = df[[s_val, c_val, c4_val, c5_val]].sum()
print(f"Total starts is {total_sums[s_val]/1000000:.2f}M or {total_sums[s_val]:.0f}")
print(f"Completion rate: {total_sums[c_val]/total_sums[s_val]*100:.1f}% with {total_sums[c_val]:.0f} completers")
print(f"4-year Completion rate: {total_sums[c4_val]/total_sums[s_val]*100:.1f}% with {total_sums[c4_val]:.0f} completers")
print(f"5-year Completion rate: {total_sums[c5_val]/total_sums[s_val]*100:.1f}% with {total_sums[c5_val]:.0f} completers")
print(f"6-year only Completion rate: {(total_sums[c_val]-total_sums[c4_val]-total_sums[c5_val])/total_sums[s_val]*100:.1f}% with {(total_sums[c_val]-total_sums[c4_val]-total_sums[c5_val]):.0f} completers")

Total starts is 0.20M or 201151
Completion rate: 55.9% with 112351 completers
4-year Completion rate: 32.8% with 65904 completers
5-year Completion rate: 17.3% with 34861 completers
6-year only Completion rate: 5.8% with 11586 completers
